In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
%%bash
wget -O - https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz | aws s3 cp - s3://api-emr-s3-1/city_market_tracker.tsv000.gz

--2024-11-20 23:42:05--  https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz
Resolving redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)... 52.92.136.50, 52.92.230.138, 3.5.86.174, ...
Connecting to redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)|52.92.136.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1315552590 (1.2G) [application/x-www-form-urlencoded]
Saving to: ‘STDOUT’

     0K .......... .......... .......... .......... ..........  0%  847K 25m16s
    50K .......... .......... .......... .......... ..........  0%  852K 25m12s
   100K .......... .......... .......... .......... ..........  0%  856K 25m8s
   150K .......... .......... .......... .......... ..........  0%  125M 18m54s
   200K .......... .......... .......... .......... ..........  0%  213M 15m8s
   250K .......... .......... .......... ........

In [4]:
spark = SparkSession.builder.appName("RedfinDataAnalysis").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
redfin_data = spark.read.csv("s3://api-emr-s3-1/city_market_tracker.tsv000.gz", header=True, inferSchema=True, sep= "\t")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
redfin_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- period_begin: date (nullable = true)
 |-- period_end: date (nullable = true)
 |-- period_duration: integer (nullable = true)
 |-- region_type: string (nullable = true)
 |-- region_type_id: integer (nullable = true)
 |-- table_id: integer (nullable = true)
 |-- is_seasonally_adjusted: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- property_type_id: integer (nullable = true)
 |-- median_sale_price: double (nullable = true)
 |-- median_sale_price_mom: double (nullable = true)
 |-- median_sale_price_yoy: double (nullable = true)
 |-- median_list_price: double (nullable = true)
 |-- median_list_price_mom: double (nullable = true)
 |-- median_list_price_yoy: double (nullable = true)
 |-- median_ppsf: double (nullable = true)
 |-- median_ppsf_mom: double (nullable = true)
 |-- median_ppsf_yoy: double (nul

In [8]:
redfin_selected = redfin_data.select(['period_end','period_duration', 'city', 'state', 'property_type',
    'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
print(redfin_selected.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5637368

In [10]:
from pyspark.sql.functions import isnull
null_counts = [redfin_selected.where(isnull(c)).count() for c in redfin_selected.columns]
null_counts

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0, 0, 0, 0, 0, 6321, 75795, 5899, 455130, 375349, 72459]

In [11]:
#better format the results
for i,col_name in  enumerate(redfin_selected.columns):
    print(f"{col_name}: {null_counts[i]} nulls")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

period_end: 0 nulls
period_duration: 0 nulls
city: 0 nulls
state: 0 nulls
property_type: 0 nulls
median_sale_price: 6321 nulls
median_ppsf: 75795 nulls
homes_sold: 5899 nulls
inventory: 455130 nulls
months_of_supply: 375349 nulls
median_dom: 72459 nulls

In [12]:
from pyspark.sql.functions import year, month

redfin_selected = redfin_selected.withColumn("period_end_yr", year(col("period_end")))
redfin_selected = redfin_selected.withColumn("period_end_month", month(col("period_end")))

redfin_selected = redfin_selected.drop("period_end")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.sql.functions import when
redfin_selected = redfin_selected.withColumn("period_end_month", 
                   when(col("period_end_month") == 1, "January")
                   .when(col("period_end_month") == 2, "February")
                   .when(col("period_end_month") == 3, "March")
                   .when(col("period_end_month") == 4, "April")
                   .when(col("period_end_month") == 5, "May")
                   .when(col("period_end_month") == 6, "June")
                   .when(col("period_end_month") == 7, "July")
                   .when(col("period_end_month") == 8, "August")
                   .when(col("period_end_month") == 9, "September")
                   .when(col("period_end_month") == 10, "October")
                   .when(col("period_end_month") == 11, "November")
                   .when(col("period_end_month") == 12, "December")
                   .otherwise("Unknown")
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
s3_bucket = 's3://api-emr-s3-1/redfin_data.parquet'
redfin_selected.write.mode("overwrite").parquet(s3_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…